# Credit Risk Resampling Techniques

## Imported Library for current project

In [10]:
import numpy as np

import pandas as pd

from pathlib import Path

from collections import Counter

import warnings

warnings.filterwarnings('ignore')

# Read the CSV into DataFrame

In [11]:
# Loaded the data

file_path = Path('Resources/lending_data.csv')

df = pd.read_csv(file_path)

df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [12]:
# Encoded features with LabelEncoder

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

le.fit(df["homeowner"])
df["homeowner"] = le.transform(df["homeowner"])



# Split the Data into Training and Testing

In [13]:
# Created features

x = df.copy()

x.drop("loan_status", axis=1, inplace=True)
# Created target

y = df['loan_status']

In [14]:
# Initialized Data

x.describe()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,0.606144,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,0.667811,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,0.000000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,0.000000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,1.000000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,1.000000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,2.000000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [16]:
# Checked the balance of our target values

y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [17]:
# Created x_train, x_test, y_train, y_test

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [18]:
# Created the StandardScaler instance

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [20]:
# Fitted the Standard Scaler with the training data

x_scaler = scaler.fit(x_train)

In [21]:
# Scaled the training and testing data

x_train_scaled = x_scaler.transform(x_train)

x_test_scaled = x_scaler.transform(x_test)

# Simple Logistic Regression

In [22]:
# Created Model for Logistic Regression 

from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)

model.fit(x_train, y_train)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score

from sklearn.metrics import balanced_accuracy_score

y_pred = model.predict(x_test)

balanced_accuracy_score(y_test, y_pred)

0.9520479254722232

In [24]:
# Displayed the confusion matrix

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[  563,    56],
       [  102, 18663]], dtype=int64)

In [25]:
# Printed the imbalanced classification report

from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      0.91      0.99      0.88      0.95      0.90       619
   low_risk       1.00      0.99      0.91      1.00      0.95      0.91     18765

avg / total       0.99      0.99      0.91      0.99      0.95      0.91     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [26]:
# Resampled the training data with the RandomOversampler

from imblearn.over_sampling import RandomOverSampler


ros = RandomOverSampler(random_state=1)

x_resampled1, y_resampled1 = ros.fit_resample(x_train, y_train)

# Viewed the count of target classes with Counter

Counter(y_resampled1)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [28]:
# Train the Logistic Regression model using the resampled data

model1 = LogisticRegression(solver='lbfgs', random_state=1)

model1.fit(x_resampled1, y_resampled1)

LogisticRegression(random_state=1)

In [29]:
# Calculated the balanced accuracy score

y_pred1 = model1.predict(x_test)

balanced_accuracy_score(y_test, y_pred1)


0.9936781215845847

In [30]:
# Displayed the confusion matrix

confusion_matrix(y_test, y_pred1)

array([[  615,     4],
       [  116, 18649]], dtype=int64)

In [31]:
# Printed the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred1, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8413    0.9935    0.9938    0.9111    0.9937    0.9874       619
   low_risk     0.9998    0.9938    0.9935    0.9968    0.9937    0.9874     18765

avg / total     0.9947    0.9938    0.9935    0.9941    0.9937    0.9874     19384



### SMOTE Oversampling

In [32]:
# Resampled the training data with SMOTE

from imblearn.over_sampling import SMOTE

x_resampled2, y_resampled2 = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(x_train, y_train)

# Viewed the count of target classes with Counter

Counter(y_resampled2)

Counter({'low_risk': 56271, 'high_risk': 56271})

In [35]:
# Train the Logistic Regression model using the resampled data

model2 = LogisticRegression(solver='lbfgs', random_state=1)

model2.fit(x_resampled2, y_resampled2)

LogisticRegression(random_state=1)

In [36]:
# Calculated the balanced accuracy score

y_pred2 = model2.predict(x_test)

balanced_accuracy_score(y_test, y_pred2)

0.9936781215845847

In [37]:
# Displayed the confusion matrix

confusion_matrix(y_test, y_pred2)

array([[  615,     4],
       [  116, 18649]], dtype=int64)

In [38]:
# Printed the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred2, digits = 4))



                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8413    0.9935    0.9938    0.9111    0.9937    0.9874       619
   low_risk     0.9998    0.9938    0.9935    0.9968    0.9937    0.9874     18765

avg / total     0.9947    0.9938    0.9935    0.9941    0.9937    0.9874     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [39]:
# Resampled the data using the ClusterCentroids resampler

from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)

x_resampled3, y_resampled3 = cc.fit_resample(x_train, y_train)

# Viewed the count of target classes with Counter

Counter(y_resampled3)

Counter({'high_risk': 1881, 'low_risk': 1881})

In [40]:
# Train the Logistic Regression model using the resampled data

model3 = LogisticRegression(solver='lbfgs', random_state=1)

model3.fit(x_resampled3, y_resampled3)

LogisticRegression(random_state=1)

In [41]:
# Calculated the balanced accuracy score

y_pred3 = model3.predict(x_test)

balanced_accuracy_score(y_test, y_pred3)

0.9881304218875842

In [42]:
# Displayed the confusion matrix

confusion_matrix(y_test, y_pred3)

array([[  608,    11],
       [  112, 18653]], dtype=int64)

In [43]:
# Printed the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred3, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8444    0.9822    0.9940    0.9081    0.9881    0.9752       619
   low_risk     0.9994    0.9940    0.9822    0.9967    0.9881    0.9775     18765

avg / total     0.9945    0.9937    0.9826    0.9939    0.9881    0.9774     19384



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [44]:
# Resampled the training data with SMOTEENN

from imblearn.combine import SMOTEENN

sm = SMOTEENN(random_state=1)

x_resampled4, y_resampled4 = sm.fit_resample(x_train, y_train)

# Viewed the count of target classes with Counter

Counter(y_resampled4)

Counter({'high_risk': 55609, 'low_risk': 55948})

In [45]:
# Train the Logistic Regression model using the resampled data

model4 = LogisticRegression(solver='lbfgs', random_state=1)

model4.fit(x_resampled4, y_resampled4)

LogisticRegression(random_state=1)

In [46]:
# Calculated the balanced accuracy score

y_pred4 = model4.predict(x_test)

balanced_accuracy_score(y_test, y_pred4)


0.9935182494822666

In [47]:
# Displayed the confusion matrix

confusion_matrix(y_test, y_pred4)

array([[  615,     4],
       [  122, 18643]], dtype=int64)

In [48]:
# Printed the imbalanced classification report

print(classification_report_imbalanced(y_test, y_pred4, digits = 4))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk     0.8345    0.9935    0.9935    0.9071    0.9935    0.9871       619
   low_risk     0.9998    0.9935    0.9935    0.9966    0.9935    0.9871     18765

avg / total     0.9945    0.9935    0.9935    0.9938    0.9935    0.9871     19384



## Final Questions

Q1. Which model had the best balanced accuracy score?

    A: Both naive over sampling and smote over sampling model has the best balanced accuracy score at 99.37% 

Q2. Which model had the best recall score?

    A: Both naive over sampling and smote over sampling model has the best balanced accuracy score at 99.38% 

Q3. Which model had the best geometric mean score?

    A: Both naive over sampling and smote over sampling model has the best geometric mean score at 99.37%.
